In [2]:
%pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
import re
import gc
import platform
import random

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as tsfm
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm
import glob

from PIL import Image

import warnings

import h5py
from PIL import Image
from io import BytesIO
warnings.filterwarnings("ignore")

In [ ]:
timm.list_models()

In [33]:
ROOT_DIR = "/kaggle/input/kostopr4vdata"
TRAIN_DIR = f'{ROOT_DIR}/images'

class CFG:
    data_path = ROOT_DIR + "/train.csv"
    train_imgs_dir = TRAIN_DIR

    img_size = 384
    fl_alpha = 1.0  
    fl_gamma = 2.0  
    cls_weight = [0.0000001, 3.0]

    cnn_name='efficientnetv2_m'
    vit_name='vit_base_patch16_384'
    seed = 69
    num_classes = 2
    batch_size = 4
    
    t_max = 16
    lr = 1e-5
    min_lr = 1e-8

    n_fold = 3
    num_workers = 8
    accum_grad_batch = 1
    early_stop_delta = 1e-7
    gpu_idx = 0
    device = torch.device(f'cuda:{gpu_idx}' if torch.cuda.is_available() else 'cpu')
    print(device)

cuda:0


In [20]:
df = pd.read_csv(CFG.data_path)
df = pd.concat([df[df.target==0].sample(frac=0.5, random_state=69), df[df.target==1].sample(frac=1, random_state=69)], axis=0)
print(df.target.value_counts())

target
0    32024
1     9239
Name: count, dtype: int64


In [22]:
import cv2
DATASET_IMAGE_MEAN = (0.485, 0.456, 0.406)
DATASET_IMAGE_STD = (0.229, 0.224, 0.225)

train_transform = A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Downscale(p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)

valid_transform =  A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)


class Dataset(Dataset):
    def __init__(self, cfg, img_names: list, labels: list, transform=None):
        self.img_dir = cfg.train_imgs_dir
        self.img_names = img_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx]+'.jpg')
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_ts = self.transform(image=img)['image']
        label_ts = self.labels[idx]
        if label_ts == 0: return img_ts, torch.tensor([1,0]), label_ts
        return img_ts, torch.tensor([0,1]), label_ts

In [23]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [47]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.backbone = timm.create_model(CFG.cnn_name, pretrained=False,
                                           num_classes=CFG.num_classes)
        try:
            in_features = self.backbone.fc.in_features
        except:
            in_features = self.backbone.classifier.in_features
        try:
            self.backbone.classifier = nn.Identity()
        except:
            pass
        self.backbone.global_pool = nn.Identity()
        try:
            self.backbone.fc = nn.Identity()
        except:
            pass
        self.pooling = GeM()
        self.dropout1 = nn.Dropout(p=0.5)
        self.linear1 = nn.Linear(1280, 640)
        self.dropout2 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(640, 320)
        self.dropout3 = nn.Dropout(p=0.2)
        self.linear3 = nn.Linear(320, 160)
        self.dropout4 = nn.Dropout(p=0.2)
        self.linear4 = nn.Linear(160, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.backbone(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear4(self.dropout4(self.linear3(self.dropout3 \
                (self.linear2(self.dropout2(self.linear1(self.dropout1(pooled_features))))))))
        output = self.softmax(output)
        return output


In [ ]:
Model()

In [34]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.backbone = timm.create_model(CFG.cnn_name, pretrained=False,
#                                            num_classes=CFG.num_classes)
#         try:
#             in_features = self.backbone.fc.in_features
#         except:
#             in_features = self.backbone.classifier.in_features
#         try:
#             self.backbone.classifier = nn.Identity()
#         except:
#             pass
#         self.backbone.global_pool = nn.Identity()
#         try:
#             self.backbone.fc = nn.Identity()
#         except:
#             pass
#         print(self.backbone)
#         self.pooling = GeM()
#         self.dropout = nn.Dropout(p=0.3)
#         self.linear = nn.Linear(in_features, CFG.num_classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, images):
#         features = self.backbone(images)
#         pooled_features = self.pooling(features).flatten(1)
#         output = self.softmax(self.linear(self.dropout(pooled_features)))
#         return output


In [38]:
if torch.cuda.is_available():
        print("Using GPU: {}\n".format(torch.cuda.get_device_name()))
        device = torch.device('cuda')
else:
        print("\nGPU not found. Using CPU: {}\n".format(platform.processor()))
        device = torch.device('cpu')

Using GPU: Tesla P100-PCIE-16GB



In [39]:
class FocalLoss(nn.Module):
    """
    The focal loss for fighting against class-imbalance
    """
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1e-12  # prevent training from Nan-loss error
        self.cls_weights = torch.tensor([CFG.cls_weight],dtype=torch.float, requires_grad=False, device=CFG.device)

    def forward(self, logits, target):
        """
        logits & target should be tensors with shape [batch_size, num_classes]
        """
        probs = torch.sigmoid(logits)
        one_subtract_probs = 1.0 - probs
        # add epsilon
        probs_new = probs + self.epsilon
        one_subtract_probs_new = one_subtract_probs + self.epsilon
        # calculate focal loss
        log_pt = target * torch.log(probs_new) + (1.0 - target) * torch.log(one_subtract_probs_new)
        pt = torch.exp(log_pt)
        focal_loss = -1.0 * (self.alpha * (1 - pt) ** self.gamma) * log_pt
        focal_loss = focal_loss * self.cls_weights
        return torch.mean(focal_loss)

In [40]:
def comp_score(v_gt, v_pred, min_tpr: float=0.80):
    
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred)#, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [44]:
def train_one_epoch(model, train_dataloader, optimizer, loss_fn, epoch, device, verbose=False):
    """
    Trains model for one epoch
    """
    model.train()

    targets_for_auroc = []
    pred_for_aucroc = []
    running_loss = 0
    prog_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    for batch, (img, for_loss, targets) in prog_bar:
        optimizer.zero_grad()
        
        img = img.to(device, torch.float)
        for_loss = for_loss.to(device, torch.float)
        targets = targets.to(device)
        
        # Re arrange the frames in the format our model wants to recieve
        preds = model(img)
        loss = loss_fn(preds, for_loss)
        
        loss.backward()
        optimizer.step()
        
        loss_item = loss.item()
        running_loss += loss_item
        
        prog_bar.set_description(f"loss: {loss_item:.4f}")
        for x in preds.cpu().detach().numpy().squeeze():
            pred_for_aucroc.append(x[1])
        for x in targets.cpu().detach().numpy():
            targets_for_auroc.append(x)        
        
        
        if verbose == True and batch % 20 == 0:
            print(f"Batch: {batch}, Loss: {loss_item}")
    
    avg_loss = running_loss / len(train_dataloader)
    print('comp_score train:',comp_score(targets_for_auroc,pred_for_aucroc))
    torch.cuda.empty_cache()
    return avg_loss

@torch.no_grad()
def valid_one_epoch(model, valid_dataloader, loss_fn,  device, verbose=False):
    """
    Validates the model for one epoch
    """
    targets_for_auroc = []
    pred_for_aucroc = []
    model.eval()
    running_loss = 0
    prog_bar = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader))
    for batch, (img, for_loss, targets) in prog_bar:
        img = img.to(device, torch.float)
        targets = targets.to(device)
        for_loss = for_loss.to(device, torch.float)
        # Re arrange the frames in the format our model wants to recieve
        preds = model(img)
        loss = loss_fn(preds, for_loss)
        
        loss_item = loss.item()
        running_loss += loss_item
        
        prog_bar.set_description(f"val_loss: {loss_item:.4f}")
        
        for x in preds.cpu().detach().numpy().squeeze():
            pred_for_aucroc.append(x[1])
        for x in targets.cpu().detach().numpy():
            targets_for_auroc.append(x)  
        
        if verbose == True and batch % 10 == 0:
            print(f"Batch: {batch}, Loss: {loss_item}")
    
    avg_val_loss = running_loss / len(valid_dataloader)
    print('comp_score val:',comp_score(targets_for_auroc,pred_for_aucroc))
    torch.cuda.empty_cache()
    
    return avg_val_loss

In [49]:
from sklearn.model_selection import KFold, StratifiedGroupKFold, StratifiedKFold
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=69)


all_img_labels_ts = train_df.target.values.tolist()
all_img_labels_val_ts = val_df.target.values.tolist()

all_img_names = train_df.isic_id.values.tolist()
all_img_names_valid = val_df.isic_id.values.tolist()


train_dataset = Dataset(CFG, all_img_names,all_img_labels_ts, train_transform)
valid_dataset = Dataset(CFG, all_img_names_valid, all_img_labels_val_ts, valid_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

model = Model().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr)
train_loss_fn = nn.BCELoss()
valid_loss_fn = nn.BCELoss()

for epoch in range(5):
    print(f"\n{'--'*8} EPOCH: {epoch+1} {'--'*8}\n")
    train_loss = train_one_epoch(model, train_loader, optimizer, train_loss_fn, epoch=epoch, device=device)
    valid_loss = valid_one_epoch(model, valid_loader, valid_loss_fn, device=device)
    
    print(f"train_loss: {train_loss:.4f}")
    print(f"val_loss: {valid_loss:.4f}")

    torch.save(model.state_dict(), f"/kaggle/working/models/model_{i}_{epoch}.pt")
    torch.cuda.empty_cache()


---------------- EPOCH: 1 ----------------



loss: 0.3683:   3%|▎         | 215/8253 [00:51<32:19,  4.14it/s]  

KeyboardInterrupt

Process Process-15:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 317, in _bootstrap
    util._exit_function()
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 334, in _exit_function
    _run_finalizers(0)
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 209, in _finalize_close
    notempty.notify()
  File "/opt/conda/lib/python3.10/threading.py", line 375, in notify
    waiter.release()
KeyboardInterrupt
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt

In [43]:
!mkdir models/

mkdir: cannot create directory 'models/': File exists
mkdir: cannot create directory 'models/': File exists


In [ ]:
# from sklearn.model_selection import KFold, StratifiedGroupKFold, StratifiedKFold

# sgkf = StratifiedKFold(n_splits=CFG.n_fold)

# for fold, ( _, val_) in enumerate(sgkf.split(df, df.target)):
#       df.loc[val_ , "kfold"] = int(fold)

# for i in range(CFG.n_fold):
#     print(f"\n{'--'*14} kfold: {i+1} {'--'*14}\n")
#     train_df = df[df['kfold']!=i]
#     val_df = df[df['kfold']==i]
#     all_img_labels_ts = train_df.target.values.tolist()
#     all_img_labels_val_ts = val_df.target.values.tolist()

#     all_img_names = train_df.isic_id.values.tolist()
#     all_img_names_valid = val_df.isic_id.values.tolist()
#     train_dataset = Dataset(CFG, all_img_names,all_img_labels_ts, train_transform)
#     valid_dataset = Dataset(CFG, all_img_names_valid, all_img_labels_val_ts, valid_transform)
#     train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
#     valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

#     model = Model().to(device)

#     optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr)
#     current_loss = 100000
#     train_loss_fn = FocalLoss()
#     valid_loss_fn = FocalLoss()
#     for epoch in range(4):
#         print(f"\n{'--'*8} EPOCH: {epoch+1} {'--'*8}\n")
                  
#         train_loss = train_one_epoch(model, train_loader, optimizer, train_loss_fn, epoch=epoch, device=device)
                  
#         valid_loss = valid_one_epoch(model, valid_loader, valid_loss_fn, device=device)
#         print(f"train_loss: {train_loss:.4f}")
#         print(f"val_loss: {valid_loss:.4f}")
                  
#         if valid_loss < current_loss:
#             current_loss = valid_loss
#             torch.save(model.state_dict(), f"/kaggle/working/models/model_{i}_{epoch}.pt")

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()